# Neural Networks

(example from sklearn and https://github.com/amirziai/deep-learning-coursera)

## The problem: determine if an image has a cat or not, given labeled data

|**Cat**| <img src="images/cat1.jpg" width="150"> || <img src="images/cat2.png" width="150"> |
|:---:|:---:|:---:|
|**Non Cat**| <img src="images/no_cat1.jpeg" width="150"> || <img src="images/no_cat2.jpg" width="150">  |

In [ ]:
import numpy as np

from sklearn import neural_network
from lr_utils import load_dataset
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# NORMALIZATION

In [ ]:
X_train = X_train_orig.reshape(X_train_orig.shape[0], -1)
X_test = X_test_orig.reshape(X_test_orig.shape[0], -1)
X_train = X_train/255.
X_test = X_test/255.

Y_train = Y_train_orig.T
Y_test = Y_test_orig.T

In [ ]:
index = 102
# print (X_train[index].reshape(()))
plt.imshow(X_train_orig[index])
print ("y = " + str(Y_train[index]) + ", it's a '" + classes[np.squeeze(Y_train[index])].decode("utf-8") +  "' picture.")

## sklearn implementation

In [ ]:
clf = neural_network.MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1, max_iter=5000)

In [ ]:
clf.fit(X_train, Y_train.ravel())

In [ ]:
predictions = clf.predict(X_train)
print ('Accuracy: %d ' % ((np.sum(Y_train.ravel() == predictions))/float(Y_train.size)*100))

In [ ]:
predictions = clf.predict(X_test)
print ('Accuracy: %d ' % ((np.sum(Y_test.ravel() == predictions))/float(Y_test.size)*100))

predictions, Y_test.ravel()

# What if we implement it?

In [ ]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

In [ ]:
def initialize_with_zeros(dim):
    w = np.zeros((dim, 1))
    b = 0.0
    return w, b

In [ ]:
def propagate(w, b, X, Y):
    m = X.shape[1]    

    # FORWARD PROPAGATION
    A = sigmoid(np.dot(w.T, X) + b)                                          # compute activation
    cost = (-1.0 / m) * np.sum(Y * np.log(A) + (1.0 - Y) * np.log(1.0 - A))  # compute cost

    # BACKWARD PROPAGATION
    dw = (1.0 / m) * np.dot(X, (A - Y).T)
    db = (1.0 / m) * np.sum(A - Y)

    cost = np.squeeze(cost)
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

In [ ]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):    
    costs = []
    
    for i in range(num_iterations):
        # Cost and gradient
        grads, cost = propagate(w, b, X, Y)
        if i % 100 == 0:
            costs.append(cost)
        dw = grads["dw"]
        db = grads["db"]
        
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [ ]:
def predict(w, b, X):
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    A = sigmoid(np.dot(w.T, X) + b)
    Y_prediction[A >= 0.5] = 1
    Y_prediction[A < 0.5] = 0
    return Y_prediction

In [ ]:
def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):

    w, b = initialize_with_zeros(X_train.shape[0])
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    w = parameters["w"]
    b = parameters["b"]
    
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)

    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [ ]:
d = model(X_train.T, Y_train.ravel(), X_test.T, Y_test.ravel(), num_iterations = 5000, learning_rate = 0.001, print_cost = True)

In [ ]:
index = 7 #30
num_px = X_train_orig[index].shape[0]

plt.imshow(X_test[index,:].reshape((num_px, num_px, 3)))
print ("y = " + str(Y_test[index,0]) + ", you predicted that it is a \"" + classes[int(d["Y_prediction_test"][0,index])].decode("utf-8") +  "\" picture.")

# Keras

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [ ]:
model = Sequential([
    Dense(32, input_shape=(X_train.shape[1],), activation="sigmoid"),
    Dropout(0.3),
    Dense(16, activation="sigmoid"),
    Dropout(0.3),
    Dense(1, activation="sigmoid")
])

In [ ]:
model.compile(
    optimizer="sgd",
    loss="binary_crossentropy",
    metrics="acc"
)
model.summary()

In [ ]:
model.fit(X_train, Y_train, batch_size=32, epochs=5, validation_data=(X_test, Y_test))